# AHAB SRK Table
The SRK table in the AHAB (Advanced High-Assurance Boot) container plays a crucial role in ensuring the integrity and authenticity of the firmware during the boot process. AHAB is a security feature that allows the microcontroller to securely authenticate and validate the firmware it loads at startup. When the microcontroller boots, the AHAB container checks the SRK table to confirm that the firmware was signed by a trusted source, providing a strong defense against unauthorized code execution and potential security breaches. The
format of the SRK table is inherited from HAB v4.x.

<img src="img/srk_table.png" alt="srk_table" width="700">

The AHAB SRK table determines the signature type and length, and consists of 1 to 4 SRK records which contains the public keys. 

<img src="img/srk_record.png" alt="srk_table" width="700">

## 1. Prerequisites
- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the [installation](../../_knowledge_base/installation_guide.rst) documentation.)

In [1]:
import os

WORKSPACE = "workspace/"  # change this to path to your workspace
DATA_DIR = os.path.join("..", "..", "_data", "keys", "ecc521")
SRK_KEYS = ["srk0_ecc521.pem", "srk1_ecc521.pem", "srk2_ecc521.pem", "srk3_ecc521.pem"]

## 2. SRK Table generation

In [2]:
from spsdk.crypto.utils import extract_public_key
from spsdk.image.ahab.ahab_srk import SRKTable
from spsdk.utils.misc import Endianness, write_file

ahab_srk = SRKTable()

for key in SRK_KEYS:
    ahab_srk.add_record(extract_public_key(f"{DATA_DIR}/{key}"))

ahab_srk.update_fields()
ahab_srk_hash = ahab_srk.compute_srk_hash()

print(ahab_srk)

# Export binary SRK table
print("SRK table in binary:")
srk_binary = ahab_srk.export()
print(srk_binary.hex())

srk_binary_path = os.path.join(WORKSPACE, "srk_table.bin")
print(f"SRK table saved to: {srk_binary_path}")
write_file(srk_binary, srk_binary_path, mode="wb")

# Display SRK hash fuses values
print("SRK hash fuses values:")
for i in range(0, len(ahab_srk_hash), 4):
    print("0x%X" % int.from_bytes(ahab_srk_hash[i : i + 4], byteorder=Endianness.LITTLE.value))

AHAB SRK table:
  Keys count:         4
  Length:             580
SRK table HASH:       53471a1098ae0ad84ac8e192f31864a5121f8963a512c9ef37ea1cb9a7a8f83a
SRK table in binary:
d7440242e1900027020300004200420000c51e858b17a8f03d637807a6085d49cf41af200ca6ac79939489528db75f4268d669f1360de5fe8bc5d8d95773221629d958dfd34528dd6ba632a7f7e616cf1e2f0101a745a331759c546ede791e80fb6231fe512e3204d8021b25be0f08d6076d8c1c56740b5eb5d0166f2219822242d66f83de347337a33fb1f55d6a690bc2651ec1e19000270203000042004200015f79416d4966e05f048d0c596ffdbf5bc763d1781ea981ac4ab067fd336788fe1ac8daa7857ad692b570d47b3a7de2c206d04f3e1d1eaece635bd4737ff0ae54aa007a8e94a15dc8aecf4af5cdc4eb4013befd262f907908ab89cb7e8bd4baf5124282533a729f66ff35f82aaccdbc74766bdb8f3a0356353efe5c24d307e0661f8d29e1900027020300004200420001e0e61acb896436c79c76235ce89a736b460763eb1bf0f039daf2f453ea1b1a32aca8e0753924f3b26948ff49c26a1183a09537f40ddf63a244e79becf523d78c9b003d224c361c0e41f362b33386067f7c7b2fd873fb799cbcf52702d763c0eee184823616865ab1adf1bc3f